# Benchmarking Different Methods 2

In this task we will benchmark a number of different methods evaluating both performance and quality 
of solutions. 

Here we continue exploration of methods.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib

font = {'family' : 'Dejavu Sans',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)


## Simple QAOA with STRAWBERRY FIELDS

In [2]:
import flatnetwork_simple as fns

In [28]:
#Setup
K = 20
dt = 1.0

#Parameters
mu = lambda t,i: 1.0
rho = 1.0

#Annealing params
J = 0.0
theta = -J*dt/K

In [29]:
L = [1,2,2]
d = 3
fnet = fns.FlatNetwork(L,d)
fnet.loadParams(mu, rho)
fnet.make_mpos()

S, D = fnet.getHamiltonian()

Max MPO dim:  9


In [30]:
print(S)
print(D)
print(np.prod(L))

{(0, 0): -0.3333333333333333, (1, 1): -1.0, (2, 2): -0.3333333333333333, (3, 3): -1.0}
{(0, 0): 0.012345679012345678, (0, 1): 0.07407407407407407, (0, 3): 0.14814814814814814, (0, 2): 0.024691358024691357, (1, 1): 0.1111111111111111, (1, 3): 0.2222222222222222, (2, 2): 0.012345679012345678, (2, 3): 0.07407407407407407, (3, 3): 0.1111111111111111}
4


In [31]:
nmodes = int(np.prod(L))
ham_simulation = sf.Program(nmodes)

with ham_simulation.context as q:
    #Prepare the initial state
    Fock(3) | q[0] #Max particles allowed in system also upper bound on particle coun
    
    for i in range(K):
        BSgate(theta, np.pi/2) | (q[0], q[1])
                        
        for pair in S:
            Rgate(-1j*S[pair]/K*dt) | q[pair[0]]
        for pair in D:
            if(pair[0] == pair[1]):
                Kgate(-1j*D[pair]/K*dt) | q[pair[0]]
            else:
                CKgate(-1j*D[pair]/K*dt) | (q[pair[0]], q[pair[1]])
        

In [32]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 3})

In [33]:
results = eng.run(ham_simulation)

In [34]:
state = results.state

for d1 in range(3):
    for d2 in range(3):
        for d3 in range(3):
            for d4 in range(3):            
                print(d1,d2,d3,d4,state.fock_prob([1, 1, 1, 0]))


0 0 0 0 0.0
0 0 0 1 0.0
0 0 0 2 0.0
0 0 1 0 0.0
0 0 1 1 0.0
0 0 1 2 0.0
0 0 2 0 0.0
0 0 2 1 0.0
0 0 2 2 0.0
0 1 0 0 0.0
0 1 0 1 0.0
0 1 0 2 0.0
0 1 1 0 0.0
0 1 1 1 0.0
0 1 1 2 0.0
0 1 2 0 0.0
0 1 2 1 0.0
0 1 2 2 0.0
0 2 0 0 0.0
0 2 0 1 0.0
0 2 0 2 0.0
0 2 1 0 0.0
0 2 1 1 0.0
0 2 1 2 0.0
0 2 2 0 0.0
0 2 2 1 0.0
0 2 2 2 0.0
1 0 0 0 0.0
1 0 0 1 0.0
1 0 0 2 0.0
1 0 1 0 0.0
1 0 1 1 0.0
1 0 1 2 0.0
1 0 2 0 0.0
1 0 2 1 0.0
1 0 2 2 0.0
1 1 0 0 0.0
1 1 0 1 0.0
1 1 0 2 0.0
1 1 1 0 0.0
1 1 1 1 0.0
1 1 1 2 0.0
1 1 2 0 0.0
1 1 2 1 0.0
1 1 2 2 0.0
1 2 0 0 0.0
1 2 0 1 0.0
1 2 0 2 0.0
1 2 1 0 0.0
1 2 1 1 0.0
1 2 1 2 0.0
1 2 2 0 0.0
1 2 2 1 0.0
1 2 2 2 0.0
2 0 0 0 0.0
2 0 0 1 0.0
2 0 0 2 0.0
2 0 1 0 0.0
2 0 1 1 0.0
2 0 1 2 0.0
2 0 2 0 0.0
2 0 2 1 0.0
2 0 2 2 0.0
2 1 0 0 0.0
2 1 0 1 0.0
2 1 0 2 0.0
2 1 1 0 0.0
2 1 1 1 0.0
2 1 1 2 0.0
2 1 2 0 0.0
2 1 2 1 0.0
2 1 2 2 0.0
2 2 0 0 0.0
2 2 0 1 0.0
2 2 0 2 0.0
2 2 1 0 0.0
2 2 1 1 0.0
2 2 1 2 0.0
2 2 2 0 0.0
2 2 2 1 0.0
2 2 2 2 0.0


In [12]:
state.all_fock_probs()

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])